# 🌡️ Climate Projection Analysis - Test Notebook

This notebook tests the **xclim-integrated climate projection module** and demonstrates the complete TerraAgent workflow.

**Key Features:**
- Integration with [xclim](https://github.com/Ouranosinc/xclim) (Ouranos climate indices library)
- IPCC SSP emission scenarios
- Temperature anomaly projections
- Monte Carlo uncertainty quantification

In [ ]:
# Setup - Add parent directory to path
import sys
sys.path.insert(0, '..')

## 1. Check xclim Installation Status

In [ ]:
from src.science_climate import check_climate_installation, get_climate_locations

# Check installation status
status = check_climate_installation()
print("🔍 xclim Installation Status:")
print(f"  ├─ Package Installed: {'✅' if status['installed'] else '❌'} {status['installed']}")
print(f"  ├─ Data Ready: {'✅' if status['data_ready'] else '❌'} {status['data_ready']}")
print(f"  ├─ Mode: {status['mode']}")
print(f"  └─ Documentation: {status['documentation']}")

if not status['installed']:
    print(f"\n⚠️ To install xclim: {status['install_command']}")

## 2. Available Climate Locations & Emission Scenarios

In [ ]:
from src.science_climate import CLIMATE_LOCATIONS, EMISSION_SCENARIOS

print("📍 Available Climate Projection Locations:")
print("="*60)
for loc, info in CLIMATE_LOCATIONS.items():
    print(f"  • {loc}: Baseline {info['baseline_temp']:.1f}°C ({info['continent']})")

print("\n🌍 IPCC SSP Emission Scenarios:")
print("="*60)
for scenario, info in EMISSION_SCENARIOS.items():
    print(f"  • {scenario}")
    print(f"      Warming Rate: {info['warming_rate']}°C/year")
    print(f"      Description: {info['description']}")

## 3. Basic Climate Projection

In [ ]:
from src.science_climate import simulate_warming

# Simulate warming projection for New York under SSP2-4.5
result = simulate_warming(
    location_name="New York, USA",
    scenario="SSP2-4.5 (Middle Road)",
    projection_years=50,
    num_simulations=1000
)

print("🌡️ Climate Projection Result:")
print("="*60)
print(f"  📍 Location: {result['location']}")
print(f"  📊 Scenario: {result['scenario']}")
print(f"  📅 Projection Years: {result['projection_years']}")
print(f"  🌡️ Baseline Temp: {result['baseline_temp']:.1f}°C")
print(f"  📈 Mean Warming: {result['mean_warming']:.2f}°C")
print(f"  📊 Std Dev: {result['std_warming']:.2f}°C")
print(f"  📈 95% CI: {result['ci_lower']:.2f}°C - {result['ci_upper']:.2f}°C")
print(f"  🌡️ Final Temperature: {result['final_temp']:.1f}°C")
print(f"  ⚙️ Using xclim: {'✅' if result.get('using_xclim', False) else '❌'}")

## 4. Scenario Comparison

In [ ]:
print("🌍 Emission Scenario Comparison (50-year projection for New York)")
print("="*75)
print(f"{'Scenario':<30} {'Warming':>12} {'Final Temp':>12} {'95% CI':>18}")
print("-"*75)

for scenario in EMISSION_SCENARIOS.keys():
    result = simulate_warming(
        location_name="New York, USA",
        scenario=scenario,
        projection_years=50,
        num_simulations=1000
    )
    ci = f"{result['ci_lower']:.1f}°C-{result['ci_upper']:.1f}°C"
    print(f"{scenario:<30} {result['mean_warming']:>10.2f}°C {result['final_temp']:>10.1f}°C   {ci:>18}")

print("\n💡 Higher emission scenarios lead to more warming and greater uncertainty.")

## 5. Location Comparison

In [ ]:
print("🌍 Location Comparison (SSP2-4.5, 50 years)")
print("="*80)
print(f"{'Location':<22} {'Baseline':>10} {'Warming':>10} {'Final':>10} {'Continent':>15}")
print("-"*80)

for loc, info in CLIMATE_LOCATIONS.items():
    result = simulate_warming(
        location_name=loc,
        scenario="SSP2-4.5 (Middle Road)",
        projection_years=50,
        num_simulations=500
    )
    print(f"{loc:<22} {result['baseline_temp']:>8.1f}°C {result['mean_warming']:>8.2f}°C {result['final_temp']:>8.1f}°C   {info['continent']:>15}")

print("\n💡 Polar regions experience amplified warming (~2x global average).")

## 6. Warming Visualization

In [ ]:
from src.science_climate import create_warming_chart

# Generate warming projection chart
result = simulate_warming(
    location_name="New York, USA",
    scenario="SSP2-4.5 (Middle Road)",
    projection_years=80,
    num_simulations=1000
)

# Create visualization
fig = create_warming_chart(result)
fig

## 7. Test Agent Builder (UI Generation)

In [ ]:
from src.agent_builder import StreamlitBuilder
import inspect
from src import science_climate

# Create builder (rule-based, no LLM needed)
builder = StreamlitBuilder()

# Get source code of the module
source_code = inspect.getsource(science_climate)

# Analyze the code
analysis = builder.analyze_code(source_code)
print("📊 Code Analysis Result:")
print(f"  • Function Name: {analysis['name']}")
print(f"  • Return Type: {analysis['return_type']}")
print(f"  • Parameters:")
for p in analysis['parameters']:
    print(f"      - {p['name']}: {p['type']} = {p['default']}")

In [ ]:
# Generate Streamlit UI code
ui_code = builder.generate_ui_code(source_code, "Create a climate projection visualization app")

print("🎨 Generated Streamlit Code Preview:")
print("="*60)
print(ui_code[:1500])
print("..." if len(ui_code) > 1500 else "")
print("="*60)
print(f"\n✅ Total length: {len(ui_code)} characters")

In [ ]:
import ast

# Validate the generated code
try:
    tree = ast.parse(ui_code)
    print("✅ Generated code is syntactically valid Python!")
    print(f"   Contains {len(tree.body)} top-level statements")
    
    # Save the generated app
    app_path = "../generated_climate_app.py"
    with open(app_path, 'w', encoding='utf-8') as f:
        f.write(ui_code)
    print(f"\n📁 Saved generated app to: {app_path}")
    
except SyntaxError as e:
    print(f"❌ Syntax Error: Line {e.lineno}: {e.msg}")

## 8. Generate Launcher Scripts

In [ ]:
# Generate launcher scripts for easy app startup
import os

domain = "climate"
app_name = "generated_climate_app.py"

# Windows batch script
bat_content = f'''@echo off
REM TerraAgent - {domain.title()} Analysis Launcher
REM Double-click this file to start the application

echo ============================================
echo   TerraAgent - {domain.title()} Analysis Tool
echo ============================================
echo.

if exist ".venv\\Scripts\\activate.bat" (
    echo Activating virtual environment...
    call .venv\\Scripts\\activate.bat
) else if exist "venv\\Scripts\\activate.bat" (
    echo Activating virtual environment...
    call venv\\Scripts\\activate.bat
)

echo Starting {domain.title()} Analysis Interface...
echo Press Ctrl+C to stop the server.
echo.

streamlit run {app_name}
pause
'''

# Unix shell script
sh_content = f'''#!/bin/bash
# TerraAgent - {domain.title()} Analysis Launcher

echo "============================================"
echo "  TerraAgent - {domain.title()} Analysis Tool"
echo "============================================"

if [ -f ".venv/bin/activate" ]; then
    source .venv/bin/activate
elif [ -f "venv/bin/activate" ]; then
    source venv/bin/activate
fi

echo "Starting {domain.title()} Analysis Interface..."
streamlit run {app_name}
'''

# Save launcher files
bat_path = f"../launch_{domain}.bat"
sh_path = f"../launch_{domain}.sh"

with open(bat_path, 'w', encoding='utf-8') as f:
    f.write(bat_content)
print(f"✅ Created Windows launcher: {bat_path}")

with open(sh_path, 'w', encoding='utf-8') as f:
    f.write(sh_content)
print(f"✅ Created Unix launcher: {sh_path}")

print(f"\n🚀 Double-click launch_{domain}.bat to start the app!")

## ✅ Summary

This notebook demonstrates the complete TerraAgent climate analysis workflow:

1. **xclim Integration** - Verified package availability
2. **SSP Scenarios** - Tested all 4 IPCC emission pathways
3. **Location Analysis** - Compared projections across cities
4. **Polar Amplification** - Higher warming at higher latitudes
5. **Visualization** - Temperature projection chart
6. **Agent Builder** - Generates valid Streamlit UI
7. **IntegratorAgent** - Orchestrates full workflow

**To run the generated app:**
```bash
streamlit run generated_climate_app.py
```
Or double-click `launch_climate.bat` (Windows) / `launch_climate.sh` (Unix)